## 작업형 제 2 유형

### ✏️ 통신사에서 고객에게 처구될 총 금액을 예측하시오.
- 제공된 데이터 목록 : churn_train.csv(훈련 데이터), churn_test.csv(평가용 데이터)

- 예측할 컬럼 : TotalCharges(총 청구액)

- 학습용 데이터(churn_train.csv)를 이용하여 총 청구액을 예측하는 모델을 만든 후 이를 평가용 데이터(churn_test.csv)에 적용해 얻은 예측값을 다음과 같은 형식의 csv 파일로 생성하시오

    - 제출 파일은 다음 1개의 컬럼을 포함해야 한다

        - pred : 예측된 총 청구액

        - 제출 파일명 : result.csv

    - 제출 한 모델의 성능은 MAE(Mean Absolute Error) 평가지표에 따라 채점한다


In [1]:
# 1, 문제 정의
# - 평가 : MAE
# - target = TotalCharges
# - 최종 파일 : result.csv

# 2. 라이브러리 및 데이터 불러오기
import pandas as pd
train = pd.read_csv('./data/churn_train.csv')
test = pd.read_csv('./data/churn_test.csv')

In [2]:
# 3. 탐색적 데이터 분석
## 데이터 크기
print(train.shape, test.shape, '\n')    # test 데이터의 행 수 1764 -> 마지막 최종 예측 값 행 수가 1764 인지 확인

## 데이터 샘플
print(train.head(), '\n')
print(test.head(), '\n')

## 데이터 자료형(타입)
print(train.info(), '\n')

## 기술 통계
print(train.describe(), '\n')
## target 기술 통계
print(train['TotalCharges'].describe(), '\n')

## object unique 개수
print(train.describe(include='O'), '\n')    # customerID 는 컬럼수 == 카테고리수 : 큰 의미 없음
print(train['customerID'].nunique(), '\n')

# 결측치 수
print(train.isnull().sum().sum(), '\n')
print(test.isnull().sum().sum(), '\n')

(4116, 19) (1764, 18) 

  customerID  gender  SeniorCitizen Partner Dependents  tenure PhoneService  \
0   CUST0454    Male              0      No         No       7          Yes   
1   CUST1145  Female              1      No        Yes      53           No   
2   CUST1138  Female              1      No        Yes      68           No   
3   CUST2645    Male              1      No        Yes      44           No   
4   CUST2632    Male              0     Yes         No       7          Yes   

      MultipleLines InternetService       OnlineSecurity         OnlineBackup  \
0               Yes              No  No internet service  No internet service   
1  No phone service     Fiber optic                  Yes                  Yes   
2  No phone service             DSL                  Yes                  Yes   
3  No phone service              No  No internet service  No internet service   
4               Yes     Fiber optic                   No                  Yes   

      DevicePr

In [3]:
# 4. 데이터 전처리
## customerID 컬럼은 제거하고 TotalCharges 컬럼은 target 변수로 옮기기
train = train.drop('customerID', axis=1)
test = test.drop('customerID', axis=1)

target = train.pop('TotalCharges')

## 원-핫 인코딩
print(train.shape, test.shape)
train = pd.get_dummies(train)
test = pd.get_dummies(test)
print(train.shape, test.shape)

(4116, 17) (1764, 17)
(4116, 43) (1764, 43)


In [4]:
# 5. 검증 데이터 분리
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(train, target, test_size=0.2, random_state=0)
print(X_train.shape, X_val.shape, y_train.shape, y_val.shape)

(3292, 43) (824, 43) (3292,) (824,)


In [5]:
# 6. 모델 학습 및 평가
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
!pip install xgboost
import xgboost as xgb
import lightgbm as lgb
from sklearn.metrics import mean_absolute_error as mae

# 선형회귀 -> 채택
lr = LinearRegression()
lr.fit(X_train, y_train)
pred = lr.predict(X_val)
print('선형 회귀 :', mae(y_val, pred))

# 랜덤포레스트
rf = RandomForestRegressor(random_state=0)
rf.fit(X_train, y_train)
pred = rf.predict(X_val)
print('랜덤 포레스트 :', mae(y_val, pred))

# xgboost
xg = xgb.XGBRegressor(random_state=0)
xg.fit(X_train, y_train)
pred = xg.predict(X_val)
print('xgboost :', mae(y_val, pred))

# lightGBM
lg = lgb.LGBMRegressor(random_state=0)
lg.fit(X_train, y_train)
pred = lg.predict(X_val)
print('lightGBM :', mae(y_val, pred))

선형 회귀 : 914.6725879047845
랜덤 포레스트 : 946.9158838362998
xgboost : 1033.3863728784358
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000186 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 157
[LightGBM] [Info] Number of data points in the train set: 3292, number of used features: 43
[LightGBM] [Info] Start training from score 2562.506563
lightGBM : 946.7024808036148


In [6]:
# 7. 최종 제출 파일
pred = lr.predict(test)
submit = pd.DataFrame({'pred':pred})
submit.to_csv('result.csv', index=False)

print(pred.shape)
print(pd.read_csv('result.csv').head())

(1764,)
          pred
0  3367.057026
1   908.379601
2  3978.660276
3   840.686927
4  1248.614397


In [ ]:
# 8-1 성능 개선
## 원-핫 인코딩(914.6725879047845) -> 레이블인코딩(913.063265926782) 채택

# 데이터 불러오기
import pandas as pd
train = pd.read_csv('./data/churn_train.csv')
test = pd.read_csv('./data/churn_test.csv')

# 데이터 전처리
train = train.drop('customerID', axis=1)
test = test.drop('customerID', axis=1)
target = train.pop('TotalCharges')

# 레이블 인코딩
print(train.shape, test.shape)
from sklearn.preprocessing import LabelEncoder
cols = train.select_dtypes(include='object').columns
for col in cols:
    le = LabelEncoder()
    train[col] = le.fit_transform(train[col])
    test[col] = le.transform(test[col])
print(train.shape, test.shape)

# 검증 데이터 분리
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(train, target, test_size=0.2, random_state=0)
print(X_train.shape, y_train.shape, X_val.shape, y_val.shape)

# 모델 학습 및 평가
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error
lr = LinearRegression()
lr.fit(X_train, y_train)
pred = lr.predict(X_val)
print(mae(y_val, pred))

(4116, 17) (1764, 17)
(4116, 17) (1764, 17)
(3292, 17) (3292,) (824, 17) (824,)
913.063265926782


In [12]:
# 8-2 성능 개선
## 스케일링 전(913.063265926782) -> 스케일링 후(913.0632659267816) 차이 없으므로 생략

# 데이터 불러오기
import pandas as pd
train = pd.read_csv('./data/churn_train.csv')
test = pd.read_csv('./data/churn_test.csv')

# 데이터 전처리
train = train.drop('customerID', axis=1)
test = test.drop('customerID', axis=1)
target = train.pop('TotalCharges')

# # 스케일링
# from sklearn.preprocessing import StandardScaler    # 913.0632659267816
# from sklearn.preprocessing import MinMaxScaler      # 913.0632659267819
# from sklearn.preprocessing import RobustScaler      # 913.0632659267816
# scaler = RobustScaler()
# cols = train.select_dtypes(exclude='object').columns
# train[cols] = scaler.fit_transform(train[cols])
# test[cols] = scaler.transform(test[cols])

# 레이블 인코딩
print(train.shape, test.shape)
from sklearn.preprocessing import LabelEncoder
cols = train.select_dtypes(include='object').columns
for col in cols:
    le = LabelEncoder()
    train[col] = le.fit_transform(train[col])
    test[col] = le.transform(test[col])
print(train.shape, test.shape)

# 검증 데이터 분리
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(train, target, test_size=0.2, random_state=0)
print(X_train.shape, y_train.shape, X_val.shape, y_val.shape)

# 모델 학습 및 평가
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error
lr = LinearRegression()
lr.fit(X_train, y_train)
pred = lr.predict(X_val)
print(mae(y_val, pred))

(4116, 17) (1764, 17)
(4116, 17) (1764, 17)
(3292, 17) (3292,) (824, 17) (824,)
913.063265926782


In [14]:
# 9. 최종 파일 제출
pred = lr.predict(test)
submit = pd.DataFrame({'pred':pred})
submit.to_csv('result.csv', index=False)

print(pred.shape)
print(pd.read_csv('result.csv').head())

(1764,)
          pred
0  3470.604235
1   931.458713
2  4014.448582
3   793.549973
4  1301.357416
